# Easy Parallelization Class in Python 

This small iPython Notebook should demonstrate my Python Parallelization Class.

First we import some neccessary packages to create an example task to be solved.

In [1]:
import numpy as np
import time

We want to solve 500 different equations of the type Ax=b. Whereby A has a shape of [500, 500] and b has [500,].

In [2]:
size = 350
number_voxels = 500

To do so we first create arrays of the size [500, 500, 250] and [500, 250].

In [3]:
A = np.random.rand(size, size, number_voxels)
b = np.random.rand(size, number_voxels)

And split them up afterwards into a list of 250 elements each.

In [4]:
As = np.dsplit(A, A.shape[-1])
bs = np.hsplit(b, b.shape[-1])

Then we start apply the numpy solving in linear order and stop the time needed.

In [11]:
timer_start = time.time()

x_sequence = []
for i in range(number_voxels):
    x_sequence.append(np.linalg.solve(np.squeeze(As[i]), np.squeeze(bs[i])))

print "Time needed: %.2f sec" % (time.time() - timer_start)

Time needed: 12.1665 sec


In order to use the parallelized solution we first have to import the parallelization class.

In [6]:
from parallelization import *

Then we have to write a function that we want to parallize. The function must be implemented for one step only. For example one voxel if we parallalize by voxel or one equation if we parallalize by equation.

In [12]:
def solve_equation(A, b):
    x = np.linalg.solve(np.squeeze(A), np.squeeze(b))
    return x

Next we create an instance of the parallalization class. This class has different parameter like display which shows the current progress of the calculation in percent, or maximum_number_of_cores, which can specify the number of CPUs to parallalize on. Default is to parallalize on all available.

In [8]:
p = parallelization(display=False)

Now we just need to call the start() function of the parallization object on give it the function to run on each CPU, as well as the number of parallization steps and every parameter the parallalized function needs.

In [13]:
timer_start = time.time()
x_parallel  = p.start(solve_equation, number_voxels, As, bs)
print "Time needed: %.2f sec" % (time.time() - timer_start)

Time needed: 2.0394 sec


The returned value will be a list of length n.

Now we compare both results of the linear and parallalized calculation and see that they are equal.

In [10]:
np.array_equal(x_sequence, x_parallel)

True